In [ ]:
import coremltools
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from coremltools.converters import keras as keras_converter
from coremltools.models.neural_network import datatypes, SgdParams
from coremltools.models import MLModel
import numpy as np

In [ ]:
def create_keras_base_model(url):
    """This method creates a convolutional neural network model using Keras.
    url - The URL that the keras model will be saved as h5 file.
    """  
    keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(6, input_dim=3, activation='relu'))
    model.add(Dense(5, activation='linear'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=0.01),
                  metrics=['accuracy'])

    model.save(url)

keras_model_path = './KerasNN.h5'
create_keras_base_model(keras_model_path)

In [ ]:
def convert_keras_to_mlmodel(keras_url, mlmodel_url):
    """This method simply converts the keras model to a mlmodel using coremltools.
    keras_url - The URL the keras model will be loaded.
    mlmodel_url - the URL the Core ML model will be saved.
    """
    keras_model = load_model(keras_url)
    mlmodel = keras_converter.convert(keras_model)
    mlmodel.save(mlmodel_url)
    return mlmodel
     
coreml_model_path = './KerasNN.mlmodel'
NNmodel = convert_keras_to_mlmodel(keras_model_path , coreml_model_path)

In [ ]:
print(NNmodel)

In [ ]:
spec = coremltools.utils.load_spec(coreml_model_path)
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.inspect_layers()
builder.inspect_input_features()

neuralnetwork_spec = builder.spec

# Set input and output description
neuralnetwork_spec.description.input[0].shortDescription = '3 Pollen Index Input'
neuralnetwork_spec.description.output[0].shortDescription = 'Predicted Symptom severity from 0-5 inclusive'

# Provide metadata
neuralnetwork_spec.description.metadata.author = 'Core ML Tools'
neuralnetwork_spec.description.metadata.license = 'MIT'
neuralnetwork_spec.description.metadata.shortDescription = (
        'An updatable neural network for predicting symptom severity')

In [ ]:
def make_updatable(builder, mlmodel_url, mlmodel_updatable_path):
    """This method makes an existing non-updatable mlmodel updatable.
    mlmodel_url - the path the Core ML model is stored.
    mlmodel_updatable_path - the path the updatable Core ML model will be saved.
    """

    model_spec = builder.spec

    # make_updatable method is used to make a layer updatable. It requires a list of layer names.
    # dense_1 and dense_2 are two innerProduct layer in this example and we make them updatable.
    builder.make_updatable(['dense_2'])

    # Categorical Cross Entropy or Mean Squared Error can be chosen for the loss layer.
    # Categorical Cross Entropy is used on this example. CCE requires two inputs: 'name' and 'input'.
    # name must be a string and will be the name associated with the loss layer
    # input must be the output of a softmax layer in the case of CCE. 
    # The loss's target will be provided automatically as a part of the model's training inputs.
    builder.set_mean_squared_error_loss(name='lossLayer', input_feature=("output1",datatypes.Array(5)))

    # in addition of the loss layer, an optimizer must also be defined. SGD and Adam optimizers are supported.
    # SGD has been used for this example. To use SGD, one must set lr(learningRate) and batch(miniBatchSize) (momentum is an optional parameter).
    builder.set_sgd_optimizer(SgdParams(lr=0.01, batch=4))

    # Finally, the number of epochs must be set as follows.
    builder.set_epochs(10)
        
    # Set training inputs descriptions
    model_spec.description.trainingInput[0].shortDescription = 'Example of pollen levels'
    model_spec.description.trainingInput[1].shortDescription = 'Associated symptom severities'

    # save the updated spec
    mlmodel_updatable = MLModel(model_spec)
    mlmodel_updatable.save(mlmodel_updatable_path)

coreml_updatable_model_path = './UpdatableKerasNN.mlmodel'
make_updatable(builder, coreml_model_path, coreml_updatable_model_path)

In [ ]:
# let's inspect the loss layer of the Core ML model
spec = coremltools.utils.load_spec(coreml_updatable_model_path)
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)

builder.inspect_loss_layers()

builder.inspect_optimizer()

# let's see which layers are updatable
builder.inspect_updatable_layers()

In [ ]:
print(builder.spec)

In [ ]:
model =  MLModel('UpdatableKerasNN.mlmodel')
predictions = model.predict(data = {'input1': np.array([3,50,100]) })

In [ ]:
print(predictions)

In [ ]:
print("done")